In [1]:
import os
import numpy as np
import random
import tensorflow as tf
import yaml

from unet_part import UNet, PCL

from utils import convert_array_to_tensor, convert_tensor_to_array, getflow, get2orderderivative

ImportError: Traceback (most recent call last):
  File "/Users/yusuke/tensorflow_macos_venv/lib/python3.8/site-packages/tensorflow/python/pywrap_tensorflow.py", line 64, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: dlopen(/Users/yusuke/tensorflow_macos_venv/lib/python3.8/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so, 6): no suitable image found.  Did find:
	/Users/yusuke/tensorflow_macos_venv/lib/python3.8/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so: mach-o, but wrong architecture
	/Users/yusuke/tensorflow_macos_venv/lib/python3.8/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so: mach-o, but wrong architecture


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

## 2D - Heat equation

### Diffusion equation
\begin{align*}
\frac{ \partial u}{\partial t}=\kappa\left(\frac{\partial^2u}{\partial x^2} + \frac{\partial^2u}{\partial y^2}\right)\ \ \ 
(0\leq x \leq x_h ,0\leq y \leq y_h , 0 \leq t)
\end{align*}

### Bounary condition
\begin{align*}
u(x,0,t)=0 \ \ \ (0\leq x \leq x_h) \\
u(x,y_h,t)=T_0 \ \ \ (0\leq x \leq x_h) \\
u(0,y,t)=0 \ \ \ (0\leq y \leq y_h) \\
u(x_h,y,t)=0 \ \ \ (0\leq y \leq y_h)
\end{align*}

## discritisation
\begin{align*}
\frac{u(x, y, t+\Delta t)-u(x, y, t)}{\Delta t} = \kappa \left(\frac{u(x+\Delta x, y, t) - 2 u(x, y, t) + u(x-\Delta x, y, t)}{\Delta x^2} + \frac{u(x, y + \Delta y, t) - 2 u(x, y, t) + u(x, y - \Delta y, t)}{\Delta y^2}\right)
\end{align*}

## Define Physics-Constrained Loss
\begin{align*}
L_d = \left|\frac{ \partial u}{\partial t}-\kappa\left(\frac{\partial^2u}{\partial x^2} + \frac{\partial^2u}{\partial y^2}\right)\right|^{2}\ \ \ 
(0\leq x \leq 1 ,0\leq y \leq 3 , 0 \leq t)
\end{align*}


\begin{align*}
L_b=\left|u(x,0,t)\right|^2+\left|u(x,y_h,t)-T_0\right|^2+\left|u(0,y,t)\right|^2+\left|u(x_h,y,t)\right|^2 \ \ \ 
\end{align*}

\begin{align*}
L_{all} = \alpha_d  L_d + \alpha_bL_b
\end{align*}

In [2]:
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
print(config)

{'learning_params': {'alpha_d': 100, 'alpha_b': 100}, 'simulation_params': {'kappa': 0.01, 'x_h': 4, 'y_h': 8, 'dx': 0.01, 'dy': 0.01}}


In [3]:
config["learning_params"]["alpha_d"]

100

In [4]:
x_h = config["simulation_params"]["x_h"]
y_h = config["simulation_params"]["y_h"]
dx = config["simulation_params"]["dx"]
dy = config["simulation_params"]["dy"]

init_temp = np.random.randint(0, 80, size=[int(x_h/dx), int(y_h/dy)]).astype(np.float32)

# set bounary condition
init_temp[:, -1] = config["simulation_params"]["T_0"]

temp_field = init_temp

config["simulation_params"]["field_shape"] = temp_field.shape

In [5]:
model = UNet(config)

In [ ]:
model.unsupervised_training(convert_array_to_tensor(init_temp))


Start of epoch 0
Training loss (for one batch) at epoch 0: 1823955222528.0000

Start of epoch 1
Training loss (for one batch) at epoch 1: 3897684882704302080.0000

Start of epoch 2
Training loss (for one batch) at epoch 2: 10786008203264.0000

Start of epoch 3
Training loss (for one batch) at epoch 3: 575092490240.0000

Start of epoch 4
Training loss (for one batch) at epoch 4: 111982362624.0000

Start of epoch 5
Training loss (for one batch) at epoch 5: 713962815488.0000

Start of epoch 6
Training loss (for one batch) at epoch 6: 1112307793920.0000

Start of epoch 7
Training loss (for one batch) at epoch 7: 1284181721088.0000

Start of epoch 8
Training loss (for one batch) at epoch 8: 1284820697088.0000

Start of epoch 9
Training loss (for one batch) at epoch 9: 1126004424704.0000

Start of epoch 10
Training loss (for one batch) at epoch 10: 845021970432.0000

Start of epoch 11
Training loss (for one batch) at epoch 11: 527130984448.0000

Start of epoch 12
Training loss (for one batc

In [ ]:
output = model.call(convert_array_to_tensor(init_temp))

In [ ]:
output